In [1]:
import mysql.connector as mysql
import pandas as pd
import datetime as dt
import numpy as np
from scipy import optimize

from datetime import datetime
import numpy as np


In [2]:
db = mysql.connect(host='127.0.0.1',
                              port='3307',
                              user='data',
                              password='hola1234',
                              database='pablito')

In [ ]:
query_string = "SELECT distinct a.id, " \
                    "DATE_FORMAT(registration_date,'%Y-%m-%d'), " \
                    "a.age, " \
                    "a.country_code, " \
                    "a.postal_code, " \
                    "a.city, " \
                    "b._promo_id_owncode " \
                "FROM lookiero.lake_user a " \
                    "inner join lookiero.users b " \
                        "on a.id = b._id and a.username = b.username " \
                    "inner join lookiero.lake_order c " \
                        "on c.user_id = a.id " \
                "where c.fraud_risk_auto_xo_pay2nok=0 " \
                    "AND date_checkout_done IS NOT NULL " \
                    "AND accounting_status<>'cancelled' " \
                    "AND c.date_cancelled is null "
               

In [ ]:
mycursor = db.cursor()

mycursor.execute(query_string)

dimensions = mycursor.fetchall()

dimensions = pd.DataFrame(dimensions)


In [ ]:
dimensions = dimensions.rename(columns={0: 'id', 1: 'registration_date', 2: 'age', 3:'country', 4: 'postal_code', 5: 'city', 6: 'own_code' })

dimensions # edad, país, last_price_range

In [ ]:
# We need to get out all the orders with promo code. Once we got that, we need to check if customers made a second order
# and if that order was done with a promo code or not. 

# Once we got the customers classified in this way, we can present the percentages. 

In [ ]:
# To do this, we call the orders and we order them. We put the next order in another column, so we can compare if the 
# promocode used lead to another order



In [ ]:
query_string = "select " \
                    "id, " \
                    "user_id,  " \
                    "promo_code,  " \
                    "date_created,  " \
                    "order_amount_net as aovn,  " \
                    "items_kept " \
                "from  lookiero.lake_order c " \
                "where c.fraud_risk_auto_xo_pay2nok=0  " \
                    "AND date_checkout_done IS NOT NULL  " \
                    "AND accounting_status<>'cancelled'  " \
                    "AND c.date_cancelled is null " \
                "order by user_id, date_created "

# "creation_type " \

In [ ]:
mycursor = db.cursor()

mycursor.execute(query_string)

orders = mycursor.fetchall()

orders = pd.DataFrame(orders)

In [ ]:
orders

In [ ]:
orders = orders.rename(columns={0:'order_id', 1:'user_id', 2: 'promo_code', 3: 'date_created', 4: 'aovn', 5: 'items_kept'}).reset_index()

orders['items_kept'] = orders['items_kept'].astype(int)

In [ ]:
orders_1 = orders

orders_1 = orders_1.rename(columns={'promo_code': 'next_promo_code', 'date_created': 'next_date_created', 'last_user_frec': 'next_last_user_frec', 
                                    'aovn': 'next_aovn', 'items_kept': 'next_items_kept', 'creation_type': 'next_creation_type'}).reset_index(drop = True)


In [ ]:
orders_1['index'] = orders_1['index'] - 1


In [ ]:
df = pd.merge(orders, orders_1, how = 'inner',  on=['index', 'user_id'])


In [ ]:
# Right now we are losing the customers that they did only one order and also we are not showing the last order as a row. 
# I can get the data of those by grouping by, then selecting only the ones that have been counted once.

In [ ]:
one_order = orders.groupby(['user_id'], as_index = True)['index'].count()

one_order = one_order.reset_index().rename(columns={'index': 'user_id', 'index': 'user_count'})

one_order = one_order[one_order.user_count == 1]

one_order = pd.merge(orders, one_order, how = 'inner',  on=['user_id'])

one_order['next_promo_code'] = None

one_order['next_date_created'] = None

one_order['next_aovn'] = 0 

one_order['next_items_kept'] = 0

one_order = one_order[['index', 'user_id', 'promo_code', 'date_created', 'aovn', 'items_kept', 'next_promo_code', 'next_date_created', 'next_aovn', 'next_items_kept']]

one_order

In [ ]:
df = df.append(one_order)

In [ ]:
df.columns

In [ ]:
del(one_order)

In [ ]:
# I need to solve how to get the last box. How, in the groping I need to get the counting > 1, and get the max date and that 
# the difference is not higher than let's say one year

In [ ]:
last_order = orders.groupby(['user_id'], as_index = True)['index'].count()

last_order = last_order.reset_index().rename(columns={'index': 'user_id', 'index': 'user_count'})

last_order = last_order[last_order.user_count > 1]

last_order = pd.merge(orders, last_order, how = 'inner',  on=['user_id'])

last_order = last_order.groupby(['user_id'], as_index = False)['date_created'].max()

last_order = pd.merge(orders, last_order, how = 'inner',  on=['user_id', 'date_created'])

last_order['next_promo_code'] = None

last_order['next_date_created'] = None

last_order['next_aovn'] = 0 

last_order['next_items_kept'] = 0

last_order = last_order[['index', 'user_id', 'promo_code', 'date_created', 'aovn', 'items_kept', 'next_promo_code', 'next_date_created',  'next_aovn', 'next_items_kept']]


In [ ]:
df = df.append(last_order)

In [ ]:
del(last_order)

In [ ]:
df = df.rename(columns={'order_id_x':'order_id', 'order_id_y':'next_order_id'})

In [ ]:
df = df[['user_id', 'order_id' , 'promo_code', 'date_created', 'aovn', 'items_kept', 'next_order_id', 'next_promo_code', 'next_date_created', 'next_aovn', 'next_items_kept']]

In [ ]:
# Ya tengo todos usuarios categorizados por su promo_code y su siguiente promocode. ¿Cómo sé si tiene un
# siguiente promocode?  # ¿Si ha pasado suficiente tiempo? o debería excluir a aquellos que su caja sea anterior a un año. 

# Puedo decir que aquellas cajas que sean 1º o últimas y haya pasado más de 6 meses



In [ ]:

df['today'] = datetime.today()

df['delta_days'] = (df['today'] - df['date_created']).dt.days

#df['next_promo_code'] = df['next_promo_code'].fillna('no_box')

#df = df[df.delta_days > 90]


In [ ]:
# Let's filter df.promocode is not null and not like LK%. In this way. To those, we will apply the number one. Ifelse, 0. we will 
# Apply the same to next_promo_code and check the correlation between both variables. 

In [ ]:
#df['promo_code'].loc[(df.promo_code.str.startswith('LK'))]

df['promo_code'] = df['promo_code'].replace('', np.nan)

df['next_promo_code'] = df['next_promo_code'].replace('', np.nan)

#df['promo_code'] = df['promo_code'].replace(None, np.nan)


# We cannot replace the none values of promocode, as it inputs the above values

In [ ]:
df.promo_code.fillna(value=np.nan, inplace=True)

df.next_promo_code.fillna(value=np.nan, inplace=True)

df = df.reset_index(drop = True)

In [ ]:
df.columns

In [ ]:
df['filter_promo_code'] = df['promo_code'].isna()

df1 = df[df.filter_promo_code == True]

df1['promo_code_corr'] = 0

df2 = df[df.filter_promo_code == False]

df2['promo_code_corr'] = 1

df = df1.append(df2)

In [ ]:
del(df1, df2)

In [ ]:
df['filter_next_promo_code'] = df['next_promo_code'].isna()

df1 = df[df.filter_next_promo_code == True]

df1['next_promo_code_corr'] = 0

df2 = df[df.filter_next_promo_code == False]

df2['next_promo_code_corr'] = 1

df = df1.append(df2)

In [ ]:
del(df1, df2)

In [ ]:
df = df[['user_id', 'order_id', 'promo_code', 'date_created', 'aovn', 'items_kept', 'next_order_id','next_promo_code', 
         'next_date_created', 'promo_code_corr', 'next_promo_code_corr', 'next_aovn', 'next_items_kept']].reset_index(drop = True)


In [ ]:
#df.groupby(['user_id'], as_index = False)['promo_code_corr'].count()

clients = df.groupby('user_id').agg({'date_created':'count', 
                                     'promo_code_corr': 'sum'}).reset_index().rename(columns={'date_created':'count',
                                                                                              'promo_code_corr':'sum_promo_code_corr'})

clients['proportion'] = clients['sum_promo_code_corr'] / clients['count']

clients = clients[['user_id', 'proportion']]

In [ ]:
df = pd.merge(df, clients, how = 'inner',  on=['user_id'])

In [ ]:
dimensions = dimensions[['id','age', 'country']].rename(columns={'id':'user_id'})

df = pd.merge(df, dimensions, how = 'inner',  on=['user_id'])

In [ ]:
age_range = pd.read_csv('age_range.csv', sep = ';')

age_range['merge_column'] = 0

In [ ]:
# Now we got by country and age (if needed) the customers that are more likely to use a promotional code for an order. 
# The need comes now to check which customers are more likely to spend more money in each of the 

In [ ]:
# The question is, if I give a promocode to a customer, will she buy the next box without a promocode? with what probability?

# the probability should be given with the historical data of her cohort or group

In [ ]:
# df['promo_code_corr'] = df.promo_code.notna()

# df.groupby(['promo_code'], as_index = True)['user_id'].count()

In [ ]:
#df.sort_values(['user_id','date_created'])

first_box = df.groupby(['user_id'], as_index = False)['date_created'].min()

first_box['first_box'] = 1

df = pd.merge(df, first_box, how = 'left',  on=['user_id', 'date_created'])

In [ ]:
df['first_box'] = df['first_box'].fillna(0)


In [ ]:
#    - Categorizar clientas si alguna vez han hecho pedidos con promo code y no se han quedado nada
# De esta forma, les ponemos un porcentaje de cajas que se han quedado gastando nada utilizando promocodes

df_no_items = df

df_no_items['promo_code'] = df_no_items['promo_code'].fillna('no_promo')

df_no_items = df_no_items[df_no_items.promo_code != 'no_promo'].reset_index(drop = True)

df_no_items_grouped = df_no_items.groupby('user_id', as_index = False).agg({'items_kept': 'mean', 'aovn': 'sum'}).reset_index(drop = True).rename(columns={'items_kept':'items_kept_promo', 'aovn': 'aovn_promo' })



In [ ]:
# https://www.datacamp.com/community/tutorials/random-forests-classifier-python

In [ ]:

df['next_date_created'] = pd.to_datetime(df['next_date_created'])

df['days_per_box'] = (df['next_date_created'] - df['date_created']).dt.days

In [ ]:
df['next_box_exist'] = np.isnat(df['next_date_created'])

df['next_box_exist'] = df['next_box_exist'].replace(False, 0)

df['next_box_exist'] = df['next_box_exist'].fillna(1)

df['next_box_exist'] = df['next_box_exist'].replace('True', 1).astype(int)

In [ ]:
############# RANDOM FOREST PER BOX

df_analysis_box = df
#df_analysis_box = pd.read_csv(df_analysis_box.csv)

#df['next_promo_code'] = df['next_promo_code'].fillna('no_promo')

In [ ]:
# nat = np.datetime64('NaT')

# df_analysis_box[df_analysis_box.next_date_created == pd.isnull(np.datetime64('NaT'))]

#df_analysis_box['next_box_exists'] = np.isnat(df_analysis_box['next_date_created'])

#df_analysis_box = df_analysis_box[df_analysis_box.next_box_exists == False]

df_analysis_box = df_analysis_box.sort_values(['user_id','date_created']).reset_index(drop = True)

#df_analysis_box['next_date_created']

In [ ]:
df_analysis_box['aovn_accumulated'] = 0
df_analysis_box['items_kept_accumulated'] = 0
len_df_analysis_box = len(df_analysis_box)

for i in range(0, len_df_analysis_box):
        if i == 0:
            df_analysis_box['aovn_accumulated'].loc[i] = df_analysis_box['aovn'].loc[i]
            df_analysis_box['items_kept_accumulated'].loc[i] = df_analysis_box['items_kept'].loc[i]
        if i > 0:
            if df_analysis_box['first_box'].loc[i] == 1:
                df_analysis_box['aovn_accumulated'].loc[i] = df_analysis_box['aovn'].loc[i]
                df_analysis_box['items_kept_accumulated'].loc[i] = df_analysis_box['items_kept'].loc[i]
            if df_analysis_box['first_box'].loc[i] == 0 and df_analysis_box['user_id'].loc[i] == df_analysis_box['user_id'].loc[i-1]:
                df_analysis_box['aovn_accumulated'].loc[i] = df_analysis_box['aovn'].loc[i] + df_analysis_box['aovn_accumulated'].loc[i - 1]
                df_analysis_box['items_kept_accumulated'].loc[i] = df_analysis_box['items_kept'].loc[i] + df_analysis_box['items_kept_accumulated'].loc[i - 1]

        

In [ ]:
df_age = dimensions[['user_id', 'age']]

df_age['merge_column'] = 0

df_age = pd.merge(df_age, age_range, how = 'left',  on=['merge_column']) 

df_age = df_age[(df_age.age >= df_age.range_min) & (df_age.age <= df_age.range_max)]

df_age = df_age[['user_id', 'age_range_id']]

df_analysis_box = pd.merge(df_age, df_analysis_box, how = 'right',  on=['user_id']) 

In [ ]:
df_analysis_box.columns

In [ ]:
# y si ordeno por cliente y fecha cada caja? De esa forma, puedo 



In [ ]:
df_country = pd.read_csv('country_id.csv', sep = ';')

df_analysis_box = pd.merge(df_analysis_box, df_country, how = 'left',  on=['country'])

In [ ]:
df_analysis_box['age_nan'] = np.isnan(df_analysis_box['age_range_id'])

df_analysis_box = df_analysis_box[(df_analysis_box.age_nan == False)].reset_index(drop = True)

df_analysis_box['age_range_id'] = df_analysis_box['age_range_id'].astype(int)


In [ ]:
df_analysis_box['aovn'] = df_analysis_box['aovn'].astype(int)

df_analysis_box['items_kept'] = df_analysis_box['items_kept'].astype(int)

df_analysis_box['aovn_accumulated'] = df_analysis_box['aovn_accumulated'].astype(int)

df_analysis_box['items_kept_accumulated'] = df_analysis_box['items_kept_accumulated'].astype(int)

df_analysis_box['promo_code_corr'] = df_analysis_box['promo_code_corr'].astype(int)

df_analysis_box['first_box'] = df_analysis_box['first_box'].astype(int)

df_analysis_box['country_id'] = df_analysis_box['country_id'].astype(int)



In [ ]:
df_analysis_box.to_csv('df_analysis_box_v1.csv')

In [3]:
df_analysis_box = pd.read_csv('df_analysis_box_v1.csv')

c:\python\lib\site-packages\IPython\core\interactiveshell.py:3155: DtypeWarning: Columns (9,10) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [ ]:
# vamos a introducir toda la siguiente info en el dataframe por caja que hemos visto que influye en el Repeat rate

In [ ]:
# RR items box one (vamos a poner los items que se ha quedado cada usuaria en su primera caja)

# channel grouping
# acquisition type

In [4]:
items_box_one = df_analysis_box[df_analysis_box.first_box == 1][['user_id', 'items_kept']].rename(columns={'items_kept':'items_kept_first_box'})

df_analysis_box = pd.merge(df_analysis_box, items_box_one, how = 'inner',  on=['user_id'])

df_analysis_box

,Unnamed: 0,user_id,age_range_id,order_id,promo_code,date_created,aovn,items_kept,next_order_id,next_promo_code,...,age,country,first_box,days_per_box,next_box_exist,aovn_accumulated,items_kept_accumulated,country_id,age_nan,items_kept_first_box
0,0,11,4,6832.0,no_promo,2016-12-08 18:20:54,98,2,18018.0,NaN,...,40.0,ES,0,102.0,0,98,2,2,False,0
1,1,11,4,18018.0,no_promo,2017-03-21 10:34:59,104,3,29777.0,NaN,...,40.0,ES,0,71.0,0,203,5,2,False,0
2,2,11,4,29777.0,no_promo,2017-05-31 18:46:56,110,3,67934.0,NaN,...,40.0,ES,0,150.0,0,314,8,2,False,0
3,3,11,4,67934.0,no_promo,2017-10-29 10:30:06,140,3,82491.0,NaN,...,40.0,ES,0,50.0,0,455,11,2,False,0
4,4,11,4,82491.0,no_promo,2017-12-18 12:32:16,220,5,189791.0,NaN,...,40.0,ES,0,196.0,0,675,16,2,False,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1410664,1409512,2792309,5,1942945.0,MIESTILO,2021-04-17 09:18:05,114,3,1961586.0,21LK10APR1,...,47.0,ES,1,7.0,0,114,3,2,False,3
1410665,1409513,2800977,7,NaN,21LK10APR1,2021-04-22 18:40:53,49,1,NaN,NaN,...,70.0,ES,0,NaN,1,0,0,2,False,2
1410666,1409514,2800977,7,1949626.0,MIESTILO,2021-04-19 18:43:52,114,2,1957070.0,21LK10APR1,...,70.0,ES,1,2.0,0,114,2,2,False,2
1410667,1409515,2804653,4,NaN,21LK10APR1,2021-04-29 15:36:39,133,4,NaN,NaN,...,44.0,ES,0,NaN,1,0,0,2,False,2


In [5]:
# channel grouping, cannot be added as we lost all history of before 2019

In [6]:
import pickle

subscriber_type = pickle.load(open("subscriber.obj", 'rb'))

occasional = pd.DataFrame(data={'user_id': [x for x, y in subscriber_type.items() if y is False],
                                'subscription_type': 1})
subscriber = pd.DataFrame(data={'user_id': [x for x, y in subscriber_type.items() if y is True],
                                'subscription_type': 2})

subscriber = occasional.append(subscriber)

In [7]:
df_analysis_box = pd.merge(df_analysis_box, subscriber, how = 'left',  on=['user_id'])


del(subscriber, occasional, subscriber_type)

In [8]:
query_string = "SELECT distinct id, " \
                    "DATE_FORMAT(registration_date,'%Y-%m-%d'), " \
                    "age, " \
                    "country_code, " \
                    "postal_code, " \
                    "city, " \
                    "newsletter_subscribed, " \
                    "last_price_range, " \
                    "nivel_aventurera, " \
                    "dedicacion, " \
                    "estilo_1, " \
                    "estilo_2 " \
                "FROM lookiero.lake_user"
                    

In [9]:
mycursor = db.cursor()

mycursor.execute(query_string)

lake_user = mycursor.fetchall()
lake_user = pd.DataFrame(lake_user)

In [10]:
lake_user = lake_user.rename(columns={0: 'user_id', 1: 'registration_date', 2: 'age', 3:'country', 4: 'postal_code',
                                     5: 'city', 6: 'newsletter_subscribed', 7: 'last_price_range', 8: 'nivel_aventurera',
                                     9: 'dedicacion', 10: 'estilo_1', 11: 'estilo_2'})

lake_user = lake_user[['user_id', 'registration_date', 'newsletter_subscribed', 'last_price_range', 'nivel_aventurera', 'dedicacion']]

In [11]:
df_analysis_box = pd.merge(df_analysis_box, lake_user, how = 'left',  on=['user_id'])


In [12]:
df_last_price_range = pd.read_csv('last_price_range_id.csv', sep = ';')

df_analysis_box = pd.merge(df_analysis_box, df_last_price_range, how = 'left',  on=['last_price_range'])

df_analysis_box

,Unnamed: 0,user_id,age_range_id,order_id,promo_code,date_created,aovn,items_kept,next_order_id,next_promo_code,...,country_id,age_nan,items_kept_first_box,subscription_type,registration_date,newsletter_subscribed,last_price_range,nivel_aventurera,dedicacion,last_price_range_id
0,0,11,4,6832.0,no_promo,2016-12-08 18:20:54,98,2,18018.0,NaN,...,2,False,0,1.0,2016-02-10,1,60-100,4.0,11.0,2.0
1,1,11,4,18018.0,no_promo,2017-03-21 10:34:59,104,3,29777.0,NaN,...,2,False,0,1.0,2016-02-10,1,60-100,4.0,11.0,2.0
2,2,11,4,29777.0,no_promo,2017-05-31 18:46:56,110,3,67934.0,NaN,...,2,False,0,1.0,2016-02-10,1,60-100,4.0,11.0,2.0
3,3,11,4,67934.0,no_promo,2017-10-29 10:30:06,140,3,82491.0,NaN,...,2,False,0,1.0,2016-02-10,1,60-100,4.0,11.0,2.0
4,4,11,4,82491.0,no_promo,2017-12-18 12:32:16,220,5,189791.0,NaN,...,2,False,0,1.0,2016-02-10,1,60-100,4.0,11.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1410664,1409512,2792309,5,1942945.0,MIESTILO,2021-04-17 09:18:05,114,3,1961586.0,21LK10APR1,...,2,False,3,1.0,2021-04-16,1,30-60,4.0,4.0,1.0
1410665,1409513,2800977,7,NaN,21LK10APR1,2021-04-22 18:40:53,49,1,NaN,NaN,...,2,False,2,1.0,2021-04-19,1,60-100,3.0,14.0,2.0
1410666,1409514,2800977,7,1949626.0,MIESTILO,2021-04-19 18:43:52,114,2,1957070.0,21LK10APR1,...,2,False,2,1.0,2021-04-19,1,60-100,3.0,14.0,2.0
1410667,1409515,2804653,4,NaN,21LK10APR1,2021-04-29 15:36:39,133,4,NaN,NaN,...,2,False,2,1.0,2021-04-21,1,30-60,3.0,2.0,1.0


In [13]:
df_analysis_box['nivel_aventurera'] = df_analysis_box['nivel_aventurera'].replace(np.nan, 0)

df_analysis_box['nivel_aventurera'] = df_analysis_box['nivel_aventurera'].astype(int)

df_analysis_box['dedicacion'] = df_analysis_box['dedicacion'].replace(np.nan, 0)

df_analysis_box['dedicacion'] = df_analysis_box['dedicacion'].astype(int)

df_analysis_box['last_price_range_id'] = df_analysis_box['last_price_range_id'].replace(np.nan, 0)

df_analysis_box['last_price_range_id'] = df_analysis_box['last_price_range_id'].astype(int)

df_analysis_box['subscription_type'] = df_analysis_box['subscription_type'].replace(np.nan, 0)

df_analysis_box['subscription_type'] = df_analysis_box['subscription_type'].astype(int)

df_analysis_box['newsletter_subscribed'] = df_analysis_box['newsletter_subscribed'].replace(np.nan, 0)

df_analysis_box['newsletter_subscribed'] = df_analysis_box['newsletter_subscribed'].astype(int)



In [14]:
df_analysis_box.columns

Index(['Unnamed: 0', 'user_id', 'age_range_id', 'order_id', 'promo_code',
       'date_created', 'aovn', 'items_kept', 'next_order_id',
       'next_promo_code', 'next_date_created', 'promo_code_corr',
       'next_promo_code_corr', 'next_aovn', 'next_items_kept', 'proportion',
       'age', 'country', 'first_box', 'days_per_box', 'next_box_exist',
       'aovn_accumulated', 'items_kept_accumulated', 'country_id', 'age_nan',
       'items_kept_first_box', 'subscription_type', 'registration_date',
       'newsletter_subscribed', 'last_price_range', 'nivel_aventurera',
       'dedicacion', 'last_price_range_id'],
      dtype='object')

In [15]:
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

# define example
data = df_analysis_box['country']
values = np.array(data)

label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(values)

onehot_encoder = OneHotEncoder(sparse=False)
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
onehot_encoded = onehot_encoder.fit_transform(integer_encoded)
print(onehot_encoded)

df_analysis_box['column_merge'] = df_analysis_box.index

onehot_encoded = pd.DataFrame(onehot_encoded)

onehot_encoded['column_merge'] = onehot_encoded.index

onehot_encoded = onehot_encoded.rename(columns={0: 'country_0', 1: 'country_1', 2: 'country_2', 3:'country_3',
                               4: 'country_4', 5: 'country_5', 6: 'country_6'})

df_analysis_box = pd.merge(df_analysis_box, onehot_encoded, how = 'inner',  on=['column_merge'])

del(data, values, onehot_encoder, integer_encoded)

[[0. 1. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 ...
 [0. 1. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]]


In [16]:
last_box = df_analysis_box[(df_analysis_box.next_box_exist == 1)]

df_analysis_box_exists = df_analysis_box[(df_analysis_box.next_box_exist == 0)]



In [17]:
df_analysis_box_exists['days_per_box'] = df_analysis_box_exists['days_per_box'].astype(int)



<ipython-input-17-f15f19732c7b>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_analysis_box_exists['days_per_box'] = df_analysis_box_exists['days_per_box'].astype(int)


In [18]:
today = dt.datetime.today()

last_box['today'] = today

last_box['date_created'] = pd.to_datetime(last_box['date_created'])

last_box['days_per_box'] = (last_box['today'] - last_box['date_created']).astype('timedelta64[D]').astype(int)

<ipython-input-18-f81dcacdada6>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  last_box['today'] = today
<ipython-input-18-f81dcacdada6>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  last_box['date_created'] = pd.to_datetime(last_box['date_created'])
<ipython-input-18-f81dcacdada6>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-d

In [ ]:
# Model to check if they will buy with promo code or not


import sklearn
from sklearn.model_selection import train_test_split

X=df_analysis_box_exists[['aovn', 'aovn_accumulated', 'items_kept', 'items_kept_accumulated', 
                   'days_per_box']] 
#PROMO_boxes, lo pongo? mejora la predicción, pero si una clienta no ha tenido nunca promos, aprende que es 0
y=df_analysis_box_exists['next_promo_code_corr']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30)

In [ ]:
from sklearn.ensemble import RandomForestClassifier

clf=RandomForestClassifier(n_estimators=50, random_state=1)


clf.fit(X_train,y_train)

y_pred = clf.predict(X_test)

In [ ]:
from sklearn import metrics
# Model Accuracy, how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))
0.89

In [ ]:
feature_columns = df_analysis_box[['aovn', 'aovn_accumulated', 'items_kept', 'items_kept_accumulated', 
                   'days_per_box']].columns

In [ ]:
feature_imp = pd.Series(clf.feature_importances_,index=
                        feature_columns ).sort_values(ascending=False)

feature_imp

In [ ]:
'''df_analysis_box[['aovn', 'aovn_accumulated', 'items_kept', 'items_kept_accumulated', 
                   'promo_code_corr', 'days_per_box', 'first_box', 'age_range_id', 'country_id', 
                   'nivel_aventurera', 'dedicacion', 'last_price_range_id', 'subscription_type', 
                   'newsletter_subscribed']].columns'''

In [ ]:
import matplotlib.pyplot as plt  
from sklearn import datasets, metrics, model_selection, svm

metrics.plot_roc_curve(clf, X_test, y_test)

In [ ]:
from sklearn.metrics import confusion_matrix

confusion_matrix(y_test, y_pred)

In [ ]:
last_box['promo_prediction'] = None

last_box = last_box.reset_index(drop = True)

len_last_box = len(last_box)

for i in range(0, len_last_box):

    prediction = clf.predict([[last_box['aovn'].loc[i], 
                  last_box['aovn_accumulated'].loc[i], 
                  last_box['items_kept'].loc[i], 
                  last_box['items_kept_accumulated'].loc[i], 
                  last_box['days_per_box'].loc[i]]])
    
    last_box.loc[i, 'promo_prediction'] = prediction
    
    

In [19]:
feature_list = df_analysis_box[['aovn', 'aovn_accumulated', 'items_kept', 'items_kept_accumulated', 
                   'promo_code_corr', 'days_per_box', 'first_box', 'age_range_id', 'country_id', 
                   'nivel_aventurera', 'dedicacion', 'last_price_range_id', 'subscription_type', 
                   'newsletter_subscribed']].columns

In [20]:
# Import tools needed for visualizationfrom sklearn.tree import export_graphviz
'''import pydot
# Pull out one tree from the forest
tree = clf.estimators_[5]
# Import tools needed for visualization
from sklearn.tree import export_graphviz
import pydot
# Pull out one tree from the forest
tree = clf.estimators_[5]
# Export the image to a dot file
export_graphviz(tree, out_file = 'tree.dot', feature_names = feature_list, rounded = True, precision = 1)
# Use dot file to create a graph
(graph, ) = pydot.graph_from_dot_file('tree.dot')
# Write graph to a png file
graph.write_png('tree.png')
'''

"import pydot\n# Pull out one tree from the forest\ntree = clf.estimators_[5]\n# Import tools needed for visualization\nfrom sklearn.tree import export_graphviz\nimport pydot\n# Pull out one tree from the forest\ntree = clf.estimators_[5]\n# Export the image to a dot file\nexport_graphviz(tree, out_file = 'tree.dot', feature_names = feature_list, rounded = True, precision = 1)\n# Use dot file to create a graph\n(graph, ) = pydot.graph_from_dot_file('tree.dot')\n# Write graph to a png file\ngraph.write_png('tree.png')\n"

In [21]:
last_box = last_box.to_csv('last_box_random_forest.csv')

In [22]:
average_spend_box = df_analysis_box



In [23]:
'''average_spend_box['today'] = datetime.today()

average_spend_box['delta_days'] = (average_spend_box['today'] - pd.to_datetime(average_spend_box['registration_date'])).dt.days


average_spend_box = average_spend_box[average_spend_box.delta_days >= 365]

average_spend_box['delta_days'] = (average_spend_box['today'] - pd.to_datetime(average_spend_box['date_created'])).dt.days

average_spend_box = average_spend_box[average_spend_box.delta_days <= 365]

'''

"average_spend_box['today'] = datetime.today()\n\naverage_spend_box['delta_days'] = (average_spend_box['today'] - pd.to_datetime(average_spend_box['registration_date'])).dt.days\n\n\naverage_spend_box = average_spend_box[average_spend_box.delta_days >= 365]\n\naverage_spend_box['delta_days'] = (average_spend_box['today'] - pd.to_datetime(average_spend_box['date_created'])).dt.days\n\naverage_spend_box = average_spend_box[average_spend_box.delta_days <= 365]\n\n"

In [24]:
df_analysis_box.columns

Index(['Unnamed: 0', 'user_id', 'age_range_id', 'order_id', 'promo_code',
       'date_created', 'aovn', 'items_kept', 'next_order_id',
       'next_promo_code', 'next_date_created', 'promo_code_corr',
       'next_promo_code_corr', 'next_aovn', 'next_items_kept', 'proportion',
       'age', 'country', 'first_box', 'days_per_box', 'next_box_exist',
       'aovn_accumulated', 'items_kept_accumulated', 'country_id', 'age_nan',
       'items_kept_first_box', 'subscription_type', 'registration_date',
       'newsletter_subscribed', 'last_price_range', 'nivel_aventurera',
       'dedicacion', 'last_price_range_id', 'column_merge', 'country_0',
       'country_1', 'country_2', 'country_3', 'country_4', 'country_5',
       'country_6'],
      dtype='object')

In [26]:
#    - Gasto medio por cajas, número de pedidos y items_kept por caja

average_spend_box = average_spend_box.groupby(['user_id', 'age_range_id',
                                               'items_kept_first_box', 'subscription_type',
                                               'newsletter_subscribed', 'last_price_range_id', 
                                               'nivel_aventurera', 'dedicacion', 'country',
                                               'country_id'], as_index = False).agg({'aovn':'sum', 'items_kept': 'sum',
                                                                                     'promo_code_corr': 'sum', 'date_created': 'count', 'days_per_box': 'mean'}).reset_index(drop = True).rename(columns={'promo_code_corr': 'promo_boxes', 'date_created': 'number_boxes'})

average_spend_box['average_aovn_box'] = average_spend_box['aovn'] / average_spend_box['number_boxes']

average_spend_box['average_items_box'] = average_spend_box['items_kept'] / average_spend_box['number_boxes']

average_spend_box

# df.groupby(['user_id'], as_index = False)['aovn', 'promo_code_corr'].sum()

,user_id,age_range_id,items_kept_first_box,subscription_type,newsletter_subscribed,last_price_range_id,nivel_aventurera,dedicacion,country,country_id,aovn,items_kept,promo_boxes,number_boxes,days_per_box,average_aovn_box,average_items_box
0,11,4,0,1,1,2,4,11,ES,2,2109,53,5,18,98.705882,117.166667,2.944444
1,41,6,5,1,1,2,3,1,ES,2,274,6,0,3,30.500000,91.333333,2.000000
2,43,6,2,1,1,3,3,3,ES,2,1788,44,2,14,116.692308,127.714286,3.142857
3,44,3,2,1,1,1,1,2,ES,2,258,6,3,5,252.250000,51.600000,1.200000
4,46,5,5,1,1,2,5,1,ES,2,143,5,1,1,NaN,143.000000,5.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
532506,2834775,4,4,1,1,1,3,14,PT,7,155,4,0,1,NaN,155.000000,4.000000
532507,2834853,3,0,2,1,1,1,3,ES,2,0,0,1,1,NaN,0.000000,0.000000
532508,2834920,3,0,2,1,2,3,4,PT,7,10,0,0,1,NaN,10.000000,0.000000
532509,2835147,3,0,1,1,1,4,13,ES,2,0,0,1,1,NaN,0.000000,0.000000


In [27]:
#clients = df_analysis_box.groupby('user_id').agg({'promo_code_corr': 'sum'}).reset_index().rename(columns={'date_created':'count', 'promo_code_corr':'promo_boxes'})

#average_spend_box = pd.merge(average_spend_box, clients, how = 'inner',  on=['user_id'])

df_no_promo = df_analysis_box[df_analysis_box.promo_code_corr == 0].reset_index(drop = True)

df_no_promo = df_no_promo.groupby('user_id', as_index = False).agg({'items_kept': 'mean', 'aovn': 'sum'}).reset_index(drop = True).rename(columns={'items_kept':'items_kept_promo', 'aovn': 'aovn_promo' })

average_spend_box = pd.merge(average_spend_box, df_no_promo, how = 'left',  on=['user_id']) 

average_spend_box['items_kept_promo'] = average_spend_box['items_kept_promo'].fillna(0)

average_spend_box['aovn_promo'] = average_spend_box['aovn_promo'].fillna(0)


In [28]:
average_spend_box

,user_id,age_range_id,items_kept_first_box,subscription_type,newsletter_subscribed,last_price_range_id,nivel_aventurera,dedicacion,country,country_id,aovn,items_kept,promo_boxes,number_boxes,days_per_box,average_aovn_box,average_items_box,items_kept_promo,aovn_promo
0,11,4,0,1,1,2,4,11,ES,2,2109,53,5,18,98.705882,117.166667,2.944444,3.230769,1621.0
1,41,6,5,1,1,2,3,1,ES,2,274,6,0,3,30.500000,91.333333,2.000000,2.000000,274.0
2,43,6,2,1,1,3,3,3,ES,2,1788,44,2,14,116.692308,127.714286,3.142857,3.250000,1555.0
3,44,3,2,1,1,1,1,2,ES,2,258,6,3,5,252.250000,51.600000,1.200000,1.500000,158.0
4,46,5,5,1,1,2,5,1,ES,2,143,5,1,1,NaN,143.000000,5.000000,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
532506,2834775,4,4,1,1,1,3,14,PT,7,155,4,0,1,NaN,155.000000,4.000000,4.000000,155.0
532507,2834853,3,0,2,1,1,1,3,ES,2,0,0,1,1,NaN,0.000000,0.000000,0.000000,0.0
532508,2834920,3,0,2,1,2,3,4,PT,7,10,0,0,1,NaN,10.000000,0.000000,0.000000,10.0
532509,2835147,3,0,1,1,1,4,13,ES,2,0,0,1,1,NaN,0.000000,0.000000,0.000000,0.0


In [29]:
average_spend_box['age_range_id'] = average_spend_box['age_range_id'].astype(int)

average_spend_box['items_kept_first_box'] = average_spend_box['items_kept_first_box'].astype(int)

average_spend_box['subscription_type'] = average_spend_box['subscription_type'].astype(int)

average_spend_box['newsletter_subscribed'] = average_spend_box['newsletter_subscribed'].astype(int)

average_spend_box['last_price_range_id'] = average_spend_box['last_price_range_id'].astype(int)

average_spend_box['nivel_aventurera'] = average_spend_box['nivel_aventurera'].astype(int)


In [30]:
average_spend_box['dedicacion'] = average_spend_box['dedicacion'].astype(int)

average_spend_box['country_id'] = average_spend_box['country_id'].astype(int)

average_spend_box['aovn'] = average_spend_box['aovn'].astype(int)

average_spend_box['items_kept'] = average_spend_box['items_kept'].astype(int)

average_spend_box['promo_boxes'] = average_spend_box['promo_boxes'].astype(int)

average_spend_box['number_boxes'] = average_spend_box['number_boxes'].astype(int)

average_spend_box['average_aovn_box'] = average_spend_box['average_aovn_box'].astype(int)

average_spend_box['average_items_box'] = average_spend_box['average_items_box'].astype(int)

average_spend_box['items_kept_promo'] = average_spend_box['items_kept_promo'].astype(int)

In [31]:
average_spend_box_model = average_spend_box

In [32]:
average_spend_box_model['average_items_box']

0         2
1         2
2         3
3         1
4         5
         ..
532506    4
532507    0
532508    0
532509    0
532510    5
Name: average_items_box, Length: 532511, dtype: int32

In [33]:
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

# define example
data = average_spend_box_model['country']
values = np.array(data)

label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(values)

onehot_encoder = OneHotEncoder(sparse=False)
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
onehot_encoded = onehot_encoder.fit_transform(integer_encoded)
print(onehot_encoded)

average_spend_box_model['column_merge'] = average_spend_box_model.index

onehot_encoded = pd.DataFrame(onehot_encoded)

onehot_encoded['column_merge'] = onehot_encoded.index

onehot_encoded = onehot_encoded.rename(columns={0: 'country_0', 1: 'country_1', 2: 'country_2', 3:'country_3',
                               4: 'country_4', 5: 'country_5', 6: 'country_6'})

average_spend_box_model = pd.merge(average_spend_box_model, onehot_encoded, how = 'inner',  on=['column_merge'])

del(data, values, onehot_encoder, integer_encoded)

[[0. 1. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 1. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]]


In [35]:
average_spend_box_model.columns

Index(['user_id', 'age_range_id', 'items_kept_first_box', 'subscription_type',
       'newsletter_subscribed', 'last_price_range_id', 'nivel_aventurera',
       'dedicacion', 'country', 'country_id', 'aovn', 'items_kept',
       'promo_boxes', 'number_boxes', 'days_per_box', 'average_aovn_box',
       'average_items_box', 'items_kept_promo', 'aovn_promo', 'column_merge',
       'country_0', 'country_1', 'country_2', 'country_3', 'country_4',
       'country_5', 'country_6'],
      dtype='object')

In [36]:
import sklearn
from sklearn.model_selection import train_test_split

X=average_spend_box_model[['items_kept_first_box', 'aovn', 'number_boxes', 'average_aovn_box']] 
#, lo pongo? mejora la predicción, pero si una clienta no ha tenido nunca promos, aprende que es 0 
# 'days_per_box', items_kept_promo
y=average_spend_box_model['average_items_box']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30)

In [37]:
from sklearn.ensemble import RandomForestClassifier

clf=RandomForestClassifier(n_estimators=50, random_state=1)



clf.fit(X_train,y_train)

y_pred=clf.predict(X_test)

In [38]:
from sklearn import metrics
# Model Accuracy, how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))

Accuracy: 0.8981621743430525
Mean Absolute Error: 0.1049175607496526
Mean Squared Error: 0.11118970417016162
Root Mean Squared Error: 0.33345120208234613


In [39]:
'''feature_columns = average_spend_box_model[['age_range_id', 'items_kept_first_box', 'subscription_type',
       'newsletter_subscribed', 'last_price_range_id', 'nivel_aventurera',
       'dedicacion', 'country_id', 'aovn', 'items_kept', 'number_boxes',
       'average_aovn_box', 'average_items_box']] .columns '''

"feature_columns = average_spend_box_model[['age_range_id', 'items_kept_first_box', 'subscription_type',\n       'newsletter_subscribed', 'last_price_range_id', 'nivel_aventurera',\n       'dedicacion', 'country_id', 'aovn', 'items_kept', 'number_boxes',\n       'average_aovn_box', 'average_items_box']] .columns "

In [40]:
feature_columns = average_spend_box_model[['items_kept_first_box', 'aovn', 'number_boxes', 'average_aovn_box']].columns

In [41]:
feature_imp = pd.Series(clf.feature_importances_,index=feature_columns).sort_values(ascending=False)

feature_imp

average_aovn_box        0.488133
items_kept_first_box    0.266689
aovn                    0.163312
number_boxes            0.081866
dtype: float64

In [42]:
from sklearn.metrics import confusion_matrix

confusion_matrix(y_test, y_pred)

array([[50819,  1240,     0,     1,     0,     0],
       [ 1393, 30762,  2068,    60,     3,     0],
       [    0,  2087, 23324,  2086,   110,     3],
       [    0,    16,  1986, 13912,  1462,   186],
       [    0,     1,    64,  1352,  5299,  1351],
       [    0,     0,     1,    38,   761, 19369]], dtype=int64)

In [43]:
['age_range_id', 'items_kept_first_box', 'subscription_type',
       'newsletter_subscribed', 'last_price_range_id', 'nivel_aventurera',
       'dedicacion', 'country_id', 'aovn', 'items_kept',
       'number_boxes', 'average_aovn_box', 'promo_boxes',
       'average_items_box']

['age_range_id',
 'items_kept_first_box',
 'subscription_type',
 'newsletter_subscribed',
 'last_price_range_id',
 'nivel_aventurera',
 'dedicacion',
 'country_id',
 'aovn',
 'items_kept',
 'number_boxes',
 'average_aovn_box',
 'promo_boxes',
 'average_items_box']

In [44]:
['items_kept_first_box', 'aovn', 'items_kept', 
 'number_boxes', 'average_aovn_box', 'average_items_box']

['items_kept_first_box',
 'aovn',
 'items_kept',
 'number_boxes',
 'average_aovn_box',
 'average_items_box']

In [45]:
average_spend_box['items_prediction'] = None

len_average_spend_box = len(average_spend_box)

for i in range(0, len_average_spend_box):

    prediction = clf.predict([[average_spend_box['items_kept_first_box'].loc[i], 
                  average_spend_box['aovn'].loc[i], 
                  average_spend_box['number_boxes'].loc[i], 
                  average_spend_box['average_aovn_box'].loc[i]]])
    
    average_spend_box.loc[i, 'items_prediction'] = prediction
    
     

In [46]:
average_spend_box.to_csv('model_item_promo_boxes.csv')